# Web scraping on Google Search + explore Top 10 words on URL's

In [15]:
# Import librairies
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [16]:
# Get data
search = 'bitcoin'
url = f'https://www.google.com/search?q={search}'
url_data = [] 

r = requests.get(url)
google = BeautifulSoup(r.text, 'lxml')
google = google.find_all('a')
url_data.append(google)
url_data = url_data[0]

In [17]:
# Create our dataframe
urls = []
texts = []
for i in url_data:
    if '/url?q=' in i['href']:
        urls.append(i['href'])
        texts.append(i.text)

start_df = pd.DataFrame({'Title': texts,'Urls' : urls})

In [18]:
# Clean url and explore them
start_df['Urls'] = start_df['Urls'].apply(lambda x: x.replace("/url?q=", ""))
start_df['Urls'] = start_df['Urls'].apply(lambda x: x.split('&', 1)[0])

In [19]:
start_df

,Title,Urls
0,Bitcoin : Un étrange afflux de requêtes via Go...,https://forex.tradingsat.com/cours-bitcoin-dol...
1,Bitcoin : Google manipulé pour booster le cour...,https://www.capital.fr/entreprises-marches/bit...
2,Bitcoin: soupçon de manipulation de cours aprè...,https://www.capital.fr/votre-argent/bitcoin-so...
3,"Bitcoin : les recherches Google explosent, pou...",https://www.lerevenu.com/bourse/devises/bitcoi...
4,30 ans de Domino's Pizza : 100 000 € à gagner ...,https://journalducoin.com/bitcoin/30-ans-de-do...
5,Facebook aurait envisagé d'utiliser Bitcoin av...,https://journalducoin.com/bitcoin/facebook-aur...
6,Le secret pour tout perdre avec les cryptomonn...,https://journalducoin.com/altcoins/hebdo-crypt...
7,L'inventeur du Bitcoin se met au rapActualités...,https://fr.cryptonews.com/news/l-inventeur-du-...
8,Confessions d'un apprenti crypto-millionnaireJ...,https://journalducoin.com/bitcoin/confessions-...
9,Qui vient de déplacer 1 milliard de dollars US...,https://fr.cryptonews.com/news/who-just-moved-...


In [20]:
# List of common words (French)
fr_words = ['de', 'le', 'des', ':', 'la', 'en', 'sur', 'les', 'et', 'du', 'pour', '?', ',', '.', 'dans', 'est', 'à', 'par',
 'une', 'qui', 'plus', 'a', ' ', 'ce', 'un', 'que', 'mes','vos', 'vous', 'au', 'aux', 'son', '']

In [21]:
# Get 10 words from our Urls
word_list = []

for url in start_df['Urls']:
    r = requests.get(url)
    r = BeautifulSoup(r.text, 'lxml')
    r = r.text.split()
    
    work_data = pd.DataFrame({'Strings':r})
    work_data['Strings'] = work_data['Strings'].apply(lambda x: re.sub(r'[^\w]', ' ', x).lower())
    work_data['Strings'] = work_data['Strings'].apply(lambda x: x.strip())
    
    idx_del = [] # Del rows with common french words (i know it exists a fct for english words, but for french ?)

    for i in range(len(work_data['Strings'])):
        if work_data['Strings'][i] in fr_words:
            idx_del.append(work_data.index[i])
        i +=1
    work_data = work_data.drop(work_data.index[idx_del], axis=0)
    
    l = work_data['Strings'].value_counts()[0:10]
    m = l.index.tolist()
    word_list.append(m)

In [24]:
pd.DataFrame(word_list).transpose()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,url,bitcoin,bitcoin,revenu,var,var,var,bitcoin,var,bitcoin,...,bitcoin,bitcoin,bitcoin,bitcoin,var,403,url,if,0,screen
1,label,google,google,bitcoin,wpcomm,wpcomm,wpcomm,actualités,wpcomm,actualités,...,sf menu,code,code,code,if,forbidden,label,bitcoin,dollar,and
2,blank,cours,cours,text,function,function,function,satoshi,function,crypto,...,318ce7,modifier,modifier,modifier,function,nginx,blank,var,solid,0
3,false,recherches,manipulation,screen,if,if,if,nakamoto,je,se,...,a hover,consulté,consulté,consulté,opt,None,false,function,euro,only
4,true,cryptomonnaie,recherches,google,0,0,0,crypto,pas,twitter,...,comment,transactions,transactions,transactions,div,None,true,const,eur,that s
5,icon,manipulation,ga set,recherches,e,bitcoin,e,twitter,if,transaction,...,td theme wrap,bitcoins,bitcoins,bitcoins,0,None,icon,bourse,var,not
6,bourse,ga set,votre,var,opt,e,opt,rap,0,communauté,...,var,2018,2018,2018,bitcoin,None,bourse,crypto monnaies,franc,arial sans serif html background fff color 2...
7,bfm,cryptomonnaies,partager,cours,div,div,div,alexander,il,usd,...,td affix,sont,sont,sont,true,None,bfm,1,and,error the
8,var,trends,favoris,avec,1,opt,1,var,1,2019,...,td header wrap,ou,ou,ou,td theme wrap,None,var,setup defaut,return,found
9,bitcoin,partager,après,if,tdc form top,1,tdc form top,hamilton,j ai,6,...,bitcoins,pas,pas,pas,1,None,bitcoin,0,usd,requested
